# Support Vector Machine Model

## Importing libraries

In [1]:
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import pandas as pd

## Reading in CSV data
Reading the dataset from a CSV into a DataFrame for use with the model.
Remove unnecessary data and split into data and labels.

In [2]:
music_df = pd.read_csv(r'D:\Uni\Y3\FYP\Code\Music Classification\extracted-features.csv')

## This will contain the data
data = music_df.iloc[:, 1:].values
## This will contain the labels corresponding to the data
labels = music_df.iloc[:, 0].values

## Create SVM Model

In [3]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size = 0.30)

svm_model = SVC(kernel="linear")

svm_model.fit(data_train, labels_train)

prediction = svm_model.predict(data_test)

print(accuracy_score(labels_test, prediction))


0.37


ValueError: could not convert string to float: 'Pop'